# Personalized Recommendation System using Agentic RAG

This project implements a personalized product recommendation system using:
- Semantic embeddings
- Vector similarity search (FAISS)
- Retrieval-Augmented Generation (RAG)
- Agent-based cart and bundle suggestions


In [16]:
#Required Libraries
!pip install sentence-transformers pandas
!pip install faiss-cpu --no-cache-dir


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [17]:
#Required Libraries
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss


In [18]:
#Load product data
df = pd.read_csv("../data/products.csv")

#Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")
#Convert product descriptions to embeddings
product_descriptions = df["description"].tolist()
product_embeddings = model.encode(product_descriptions)

print("Embedding shape:", product_embeddings.shape)

#Create FAISS index
dimension = product_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(product_embeddings)
user_query = "I need items for my workout"

#Convert query to embedding
query_embedding = model.encode([user_query])

#Similarity search
D, I = index.search(query_embedding, k=3)

print("\nTop product recommendations:")
for idx in I[0]:
    print("-", df.iloc[idx]["name"])

Embedding shape: (5, 384)

Top product recommendations:
- Gym Bag
- Running Shoes
- Protein Shaker


USER HISTORY + PERSONALIZATION (DEEP & PRACTICAL)

In [19]:
#Creating User Profile
user_profile = {
    "user_id": "U1",
    "history": "Purchased running shoes and gym bag. Interested in fitness and workouts."
}


In [20]:
user_embedding = model.encode([user_profile["history"]])
print("User embedding shape:", user_embedding.shape)


User embedding shape: (1, 384)


In [21]:
#Personalized Search (User → Products)
D, I = index.search(user_embedding, k=3)

print("Personalized Recommendations:")
for idx in I[0]:
    print("-", df.iloc[idx]["name"])


Personalized Recommendations:
- Running Shoes
- Gym Bag
- Fitness Band


RAG(Retriveal Augumented Gentration)

In [22]:
# Get top products for user
D, I = index.search(user_embedding, k=3)

retrieved_products = []
for idx in I[0]:
    product = df.iloc[idx]
    retrieved_products.append(
        f"{product['name']}: {product['description']}"
    )

retrieved_products


['Running Shoes: Lightweight shoes for daily jogging and running',
 'Gym Bag: Large gym bag to carry workout essentials',
 'Fitness Band: Tracks steps calories and heart rate']

In [24]:
rag_prompt = f"""
You are a shopping assistant.

User profile:
Interested in fitness and workouts.

Retrieved products:
{chr(10).join(retrieved_products)}

Task:
Explain why these products are recommended for the user.
"""
print(rag_prompt)



You are a shopping assistant.

User profile:
Interested in fitness and workouts.

Retrieved products:
Running Shoes: Lightweight shoes for daily jogging and running
Gym Bag: Large gym bag to carry workout essentials
Fitness Band: Tracks steps calories and heart rate

Task:
Explain why these products are recommended for the user.



In [25]:
#mock LLM function
def mock_llm(prompt):
    return """
Based on the user's interest in fitness and workouts, these products are recommended because:

- Running Shoes support daily jogging and exercise routines.
- Gym Bag helps carry essential workout equipment.
- Protein Shaker is useful for post-workout nutrition and recovery.

These items together support an active and fitness-focused lifestyle.
"""

rag_response = mock_llm(rag_prompt)
print(rag_response)




Based on the user's interest in fitness and workouts, these products are recommended because:

- Running Shoes support daily jogging and exercise routines.
- Gym Bag helps carry essential workout equipment.
- Protein Shaker is useful for post-workout nutrition and recovery.

These items together support an active and fitness-focused lifestyle.



AGENTIC LOGIC

In [26]:
#raw input
[
 'Protein Shaker: Bottle used for protein shakes and smoothies',
 'Running Shoes: Lightweight shoes for daily jogging and running',
 'Gym Bag: Large gym bag to carry workout essentials'
]


['Protein Shaker: Bottle used for protein shakes and smoothies',
 'Running Shoes: Lightweight shoes for daily jogging and running',
 'Gym Bag: Large gym bag to carry workout essentials']

In [28]:
#Agent Function

def shopping_agent(retrieved_products):
    cart = []
    suggestions = []

    product_text = " ".join(retrieved_products).lower()

    # Rule 1: If fitness items exist, suggest essentials
    if "shoes" in product_text:
        cart.append("Running Shoes")

    if "gym bag" in product_text:
        cart.append("Gym Bag")

    if "protein" in product_text:
        cart.append("Protein Shaker")

    # Rule 2: Bundle logic
    if len(cart) >= 2:
        suggestions.append(
            "Bundle Offer: Running Shoes + Gym Bag + Protein Shaker (10% off)"
        )

    # Rule 3: Upsell logic
    suggestions.append(
        "Suggested Add-on: Fitness Band to track your workouts"
    )

    return cart, suggestions


In [29]:
cart_items, agent_suggestions = shopping_agent(retrieved_products)

print("🛒 Items added to cart:")
for item in cart_items:
    print("-", item)

print("\n🤖 Agent Suggestions:")
for s in agent_suggestions:
    print("-", s)


🛒 Items added to cart:
- Running Shoes
- Gym Bag

🤖 Agent Suggestions:
- Bundle Offer: Running Shoes + Gym Bag + Protein Shaker (10% off)
- Suggested Add-on: Fitness Band to track your workouts


In [ ]:
#Final Output

print("🔍 Personalized Product Recommendations:\n")
for idx in I[0]:
    print("-", df.iloc[idx]["name"])

print("\n🧠 RAG Explanation:\n")
print(rag_response)

print("\n🛒 Agent Cart Suggestions:\n")
for item in cart_items:
    print("-", item)

print("\n🤖 Agent Offers:\n")
for s in agent_suggestions:
    print("-", s)


🔍 Personalized Product Recommendations:

- Running Shoes
- Gym Bag
- Fitness Band

🧠 RAG Explanation:


Based on the user's interest in fitness and workouts, these products are recommended because:

- Running Shoes support daily jogging and exercise routines.
- Gym Bag helps carry essential workout equipment.
- Protein Shaker is useful for post-workout nutrition and recovery.

These items together support an active and fitness-focused lifestyle.


🛒 Agent Cart Suggestions:

- Running Shoes
- Gym Bag

🤖 Agent Offers:

- Bundle Offer: Running Shoes + Gym Bag + Protein Shaker (10% off)
- Suggested Add-on: Fitness Band to track your workouts
